In [1]:
from datetime import datetime
import os

from sklearn.model_selection import KFold
from sklearn.metrics import fbeta_score
from sklearn.preprocessing import MinMaxScaler
from category_encoders.basen import BaseNEncoder
from imblearn.ensemble import BalancedRandomForestClassifier

Using TensorFlow backend.


In [5]:
import pandas as pd
pd.set_option('display.max_columns', 500)
df = pd.read_csv('dfs/df0002.csv')
df.head(50)

,EmployeeID,Date,DevCenterID,SBUID,PositionID,PositionLevel,IsTrainee,LanguageLevelID,CustomerID,ProjectID,IsInternalProject,Utilization,HourVacation,HourMobileReserve,HourLockedReserve,OnSite,CompetenceGroupID,FunctionalOfficeID,PaymentTypeId,BonusOneTime,APM,WageGross,MonthOnPosition,MonthOnSalary,HiringDate,DismissalDate,target,Utilization_mean,Utilization_std,Utilization_min,Utilization_max,Utilization_nunique,Utilization_nunique_frac,Utilization_last_m_max,Utilization_last_m_min,Utilization_last_m_mean,Utilization_time_since_lat_change,HourVacation_mean,HourVacation_std,HourVacation_min,HourVacation_max,HourVacation_nunique,HourVacation_nunique_frac,HourVacation_last_m_max,HourVacation_last_m_min,HourVacation_last_m_mean,HourVacation_time_since_lat_change,HourMobileReserve_mean,HourMobileReserve_std,HourMobileReserve_min,HourMobileReserve_max,HourMobileReserve_nunique,HourMobileReserve_nunique_frac,HourMobileReserve_last_m_max,HourMobileReserve_last_m_min,HourMobileReserve_last_m_mean,HourMobileReserve_time_since_lat_change,HourLockedReserve_mean,HourLockedReserve_std,HourLockedReserve_min,HourLockedReserve_max,HourLockedReserve_nunique,HourLockedReserve_nunique_frac,HourLockedReserve_last_m_max,HourLockedReserve_last_m_min,HourLockedReserve_last_m_mean,HourLockedReserve_time_since_lat_change,BonusOneTime_mean,BonusOneTime_std,BonusOneTime_min,BonusOneTime_max,BonusOneTime_nunique,BonusOneTime_nunique_frac,BonusOneTime_last_m_max,BonusOneTime_last_m_min,BonusOneTime_last_m_mean,BonusOneTime_time_since_lat_change,APM_mean,APM_std,APM_min,APM_max,APM_nunique,APM_nunique_frac,APM_last_m_max,APM_last_m_min,APM_last_m_mean,APM_time_since_lat_change,WageGross_mean,WageGross_std,WageGross_min,WageGross_max,WageGross_nunique,WageGross_nunique_frac,WageGross_last_m_max,WageGross_last_m_min,WageGross_last_m_mean,WageGross_time_since_lat_change
0,00116D71-E87D-4B64-A566-1F29B2A798A8,7/1/17,3,292,70,2,0,11,893EA22F-08BE-4F11-AD93-C50746E4565F,7F97465B-ED3C-45DB-BE96-86C7E493F0CD,0,0.7619,40,0,0,0,15,1,9,0,39,0.887446,1,1,6/1/13,NaN,0,0.91757,0.145803,0.5,1.0,8,0.40,0.0,0.5,0.08243,0,17.0,21.311968,0,72,8,0.40,-64,8,-9.0,0,0.0,0.0,0,0,1,0.05,0,0,0.0,0,0.0,0.0,0,0,1,0.05,0,0,0.0,0,25.0,62.249498,0,200,3,0.15,-200,0,-25.0,1,37.3,14.234114,0,49,14,0.70,-15,34,-3.3,0,0.924879,0.056801,0.887446,1.089649,4,0.20,0.0,0.202203,0.164770,0
1,00116D71-E87D-4B64-A566-1F29B2A798A8,8/1/17,3,332,70,2,0,11,893EA22F-08BE-4F11-AD93-C50746E4565F,7F97465B-ED3C-45DB-BE96-86C7E493F0CD,0,1.0000,0,0,0,0,15,1,9,200,28,0.887446,2,2,6/1/13,NaN,0,0.91757,0.145803,0.5,1.0,8,0.40,0.0,0.5,0.08243,0,17.0,21.311968,0,72,8,0.40,-64,8,-9.0,0,0.0,0.0,0,0,1,0.05,0,0,0.0,0,0.0,0.0,0,0,1,0.05,0,0,0.0,0,25.0,62.249498,0,200,3,0.15,-200,0,-25.0,1,37.3,14.234114,0,49,14,0.70,-15,34,-3.3,0,0.924879,0.056801,0.887446,1.089649,4,0.20,0.0,0.202203,0.164770,0
2,00116D71-E87D-4B64-A566-1F29B2A798A8,9/1/17,3,332,70,2,0,11,893EA22F-08BE-4F11-AD93-C50746E4565F,7F97465B-ED3C-45DB-BE96-86C7E493F0CD,0,0.5000,72,0,0,0,15,1,9,0,43,0.887446,3,3,6/1/13,NaN,0,0.91757,0.145803,0.5,1.0,8,0.40,0.0,0.5,0.08243,0,17.0,21.311968,0,72,8,0.40,-64,8,-9.0,0,0.0,0.0,0,0,1,0.05,0,0,0.0,0,0.0,0.0,0,0,1,0.05,0,0,0.0,0,25.0,62.249498,0,200,3,0.15,-200,0,-25.0,1,37.3,14.234114,0,49,14,0.70,-15,34,-3.3,0,0.924879,0.056801,0.887446,1.089649,4,0.20,0.0,0.202203,0.164770,0
3,00116D71-E87D-4B64-A566-1F29B2A798A8,10/1/17,3,332,70,2,0,11,893EA22F-08BE-4F11-AD93-C50746E4565F,7F97465B-ED3C-45DB-BE96-86C7E493F0CD,0,0.8636,16,0,0,0,15,1,9,0,49,0.887446,4,4,6/1/13,NaN,0,0.91757,0.145803,0.5,1.0,8,0.40,0.0,0.5,0.08243,0,17.0,21.311968,0,72,8,0.40,-64,8,-9.0,0,0.0,0.0,0,0,1,0.05,0,0,0.0,0,0.0,0.0,0,0,1,0.05,0,0,0.0,0,25.0,62.249498,0,200,3,0.15,-200,0,-25.0,1,37.3,14.234114,0,49,14,0.70,-15,34,-3.3,0,0.924879,0.056801,0.887446,1.089649,4,0.20,0.0,0.202203,0.164770,0
4,00116D71-E87D-4B64-A566-1F29B2A798A8,11/1/17,3,332,70,2,0,11,893EA22F-08BE-4F11-AD93-C50746E4565F,7F97465B-ED3C-45DB-BE96-86C7E493F0CD,0,0.8000,8,0,0,0,15,1,